In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup, SoupStrainer
import sys
import string
import requests
import time

In [15]:
def player_info():
    players = []
    base_url = "http://www.basketball-reference.com/players/"
    
    for letter in string.ascii_lowercase:
        page = requests.get(base_url+letter)
        soup = BeautifulSoup(page.text,features = "lxml")
        table = soup.find('tbody')
        
        if table:
            for row in table.findAll('tr'):
                print(row)
    

In [16]:
player_info()

<tr><th class="left " data-append-csv="abdelal01" data-stat="player" scope="row"><a href="/players/a/abdelal01.html">Alaa Abdelnaby</a></th><td class="right " data-stat="year_min">1991</td><td class="right " data-stat="year_max">1995</td><td class="center " data-stat="pos">F-C</td><td class="right " csk="82.0" data-stat="height">6-10</td><td class="right " data-stat="weight">240</td><td class="left " csk="19680624" data-stat="birth_date"><a href="/friv/birthdays.cgi?month=6&amp;day=24">June 24, 1968</a></td><td class="left " data-stat="colleges"><a href="/friv/colleges.fcgi?college=duke">Duke University</a></td></tr>
<tr><th class="left " data-append-csv="abdulza01" data-stat="player" scope="row"><a href="/players/a/abdulza01.html">Zaid Abdul-Aziz</a></th><td class="right " data-stat="year_min">1969</td><td class="right " data-stat="year_max">1978</td><td class="center " data-stat="pos">C-F</td><td class="right " csk="81.0" data-stat="height">6-9</td><td class="right " data-stat="weigh

<tr><th class="left " data-append-csv="babbch01" data-stat="player" scope="row"><a href="/players/b/babbch01.html">Chris Babb</a></th><td class="right " data-stat="year_min">2014</td><td class="right " data-stat="year_max">2014</td><td class="center " data-stat="pos">G</td><td class="right " csk="77.0" data-stat="height">6-5</td><td class="right " data-stat="weight">225</td><td class="left " csk="19900214" data-stat="birth_date"><a href="/friv/birthdays.cgi?month=2&amp;day=14">February 14, 1990</a></td><td class="left " data-stat="colleges"><a href="/friv/colleges.fcgi?college=pennst">Pennsylvania State University</a>, <a href="/friv/colleges.fcgi?college=iowast">Iowa State University</a></td></tr>
<tr><th class="left " data-append-csv="babbilu01" data-stat="player" scope="row"><strong><a href="/players/b/babbilu01.html">Luke Babbitt</a></strong></th><td class="right " data-stat="year_min">2011</td><td class="right " data-stat="year_max">2018</td><td class="center " data-stat="pos">F</

<tr><th class="left " data-append-csv="cabarza01" data-stat="player" scope="row"><a href="/players/c/cabarza01.html">Zarko Cabarkapa</a></th><td class="right " data-stat="year_min">2004</td><td class="right " data-stat="year_max">2006</td><td class="center " data-stat="pos">F</td><td class="right " csk="83.0" data-stat="height">6-11</td><td class="right " data-stat="weight">235</td><td class="left " csk="19810521" data-stat="birth_date"><a href="/friv/birthdays.cgi?month=5&amp;day=21">May 21, 1981</a></td><td class="left iz" data-stat="colleges"></td></tr>
<tr><th class="left " data-append-csv="cableba01" data-stat="player" scope="row"><a href="/players/c/cableba01.html">Barney Cable</a></th><td class="right " data-stat="year_min">1959</td><td class="right " data-stat="year_max">1964</td><td class="center " data-stat="pos">F</td><td class="right " csk="79.0" data-stat="height">6-7</td><td class="right " data-stat="weight">175</td><td class="left " csk="19350729" data-stat="birth_date">

<tr><th class="left " data-append-csv="dantomi01" data-stat="player" scope="row"><a href="/players/d/dantomi01.html">Mike D'Antoni</a></th><td class="right " data-stat="year_min">1974</td><td class="right " data-stat="year_max">1977</td><td class="center " data-stat="pos">G</td><td class="right " csk="75.0" data-stat="height">6-3</td><td class="right " data-stat="weight">185</td><td class="left " csk="19510508" data-stat="birth_date"><a href="/friv/birthdays.cgi?month=5&amp;day=8">May 8, 1951</a></td><td class="left " data-stat="colleges"><a href="/friv/colleges.fcgi?college=marshall">Marshall University</a></td></tr>
<tr><th class="left " data-append-csv="dabicmi01" data-stat="player" scope="row"><a href="/players/d/dabicmi01.html">Mike Dabich</a></th><td class="right " data-stat="year_min">1968</td><td class="right " data-stat="year_max">1968</td><td class="center " data-stat="pos">C</td><td class="right " csk="84.0" data-stat="height">7-0</td><td class="right " data-stat="weight">24

<tr><th class="left " data-append-csv="eacklle01" data-stat="player" scope="row"><a href="/players/e/eacklle01.html">Ledell Eackles</a></th><td class="right " data-stat="year_min">1989</td><td class="right " data-stat="year_max">1998</td><td class="center " data-stat="pos">G-F</td><td class="right " csk="77.0" data-stat="height">6-5</td><td class="right " data-stat="weight">220</td><td class="left " csk="19661124" data-stat="birth_date"><a href="/friv/birthdays.cgi?month=11&amp;day=24">November 24, 1966</a></td><td class="left " data-stat="colleges"><a href="/friv/colleges.fcgi?college=norleans">University of New Orleans</a></td></tr>
<tr><th class="left " data-append-csv="eakinji01" data-stat="player" scope="row"><a href="/players/e/eakinji01.html">Jim Eakins</a></th><td class="right " data-stat="year_min">1969</td><td class="right " data-stat="year_max">1978</td><td class="center " data-stat="pos">C</td><td class="right " csk="83.0" data-stat="height">6-11</td><td class="right " data

...

<tr><th class="left " data-append-csv="tabakza01" data-stat="player" scope="row"><a href="/players/t/tabakza01.html">Zan Tabak</a></th><td class="right " data-stat="year_min">1995</td><td class="right " data-stat="year_max">2001</td><td class="center " data-stat="pos">C</td><td class="right " csk="84.0" data-stat="height">7-0</td><td class="right " data-stat="weight">245</td><td class="left " csk="19700615" data-stat="birth_date"><a href="/friv/birthdays.cgi?month=6&amp;day=15">June 15, 1970</a></td><td class="left iz" data-stat="colleges"></td></tr>
<tr><th class="left " data-append-csv="tabusyu01" data-stat="player" scope="row"><a href="/players/t/tabusyu01.html">Yuta Tabuse</a></th><td class="right " data-stat="year_min">2005</td><td class="right " data-stat="year_max">2005</td><td class="center " data-stat="pos">G</td><td class="right " csk="69.0" data-stat="height">5-9</td><td class="right " data-stat="weight">165</td><td class="left " csk="19801005" data-stat="birth_date"><a href

<tr><th class="left " data-append-csv="ubileed01" data-stat="player" scope="row"><a href="/players/u/ubileed01.html">Edwin Ubiles</a></th><td class="right " data-stat="year_min">2012</td><td class="right " data-stat="year_max">2012</td><td class="center " data-stat="pos">G</td><td class="right " csk="78.0" data-stat="height">6-6</td><td class="right " data-stat="weight">204</td><td class="left " csk="19861126" data-stat="birth_date"><a href="/friv/birthdays.cgi?month=11&amp;day=26">November 26, 1986</a></td><td class="left " data-stat="colleges"><a href="/friv/colleges.fcgi?college=siena">Siena College</a></td></tr>
<tr><th class="left " data-append-csv="udohek01" data-stat="player" scope="row"><strong><a href="/players/u/udohek01.html">Ekpe Udoh</a></strong></th><td class="right " data-stat="year_min">2011</td><td class="right " data-stat="year_max">2019</td><td class="center " data-stat="pos">C-F</td><td class="right " csk="82.0" data-stat="height">6-10</td><td class="right " data-st

<tr><th class="left " data-append-csv="wadedw01" data-stat="player" scope="row"><strong><a href="/players/w/wadedw01.html">Dwyane Wade</a></strong></th><td class="right " data-stat="year_min">2004</td><td class="right " data-stat="year_max">2019</td><td class="center " data-stat="pos">G</td><td class="right " csk="76.0" data-stat="height">6-4</td><td class="right " data-stat="weight">220</td><td class="left " csk="19820117" data-stat="birth_date"><a href="/friv/birthdays.cgi?month=1&amp;day=17">January 17, 1982</a></td><td class="left " data-stat="colleges"><a href="/friv/colleges.fcgi?college=marquette">Marquette University</a></td></tr>
<tr><th class="left " data-append-csv="wadema01" data-stat="player" scope="row"><a href="/players/w/wadema01.html">Mark Wade</a></th><td class="right " data-stat="year_min">1988</td><td class="right " data-stat="year_max">1990</td><td class="center " data-stat="pos">G</td><td class="right " csk="71.0" data-stat="height">5-11</td><td class="right " dat

<tr><th class="left " data-append-csv="yabusgu01" data-stat="player" scope="row"><strong><a href="/players/y/yabusgu01.html">Guerschon Yabusele</a></strong></th><td class="right " data-stat="year_min">2018</td><td class="right " data-stat="year_max">2019</td><td class="center " data-stat="pos">F</td><td class="right " csk="79.0" data-stat="height">6-7</td><td class="right " data-stat="weight">260</td><td class="left " csk="19951217" data-stat="birth_date"><a href="/friv/birthdays.cgi?month=12&amp;day=17">December 17, 1995</a></td><td class="left iz" data-stat="colleges"></td></tr>
<tr><th class="left " data-append-csv="yarbrvi01" data-stat="player" scope="row"><a href="/players/y/yarbrvi01.html">Vincent Yarbrough</a></th><td class="right " data-stat="year_min">2003</td><td class="right " data-stat="year_max">2003</td><td class="center " data-stat="pos">G-F</td><td class="right " csk="79.0" data-stat="height">6-7</td><td class="right " data-stat="weight">210</td><td class="left " csk="1

<tr><th class="left " data-append-csv="zasloma01" data-stat="player" scope="row"><a href="/players/z/zasloma01.html">Max Zaslofsky</a></th><td class="right " data-stat="year_min">1947</td><td class="right " data-stat="year_max">1956</td><td class="center " data-stat="pos">G-F</td><td class="right " csk="74.0" data-stat="height">6-2</td><td class="right " data-stat="weight">170</td><td class="left " csk="19251207" data-stat="birth_date"><a href="/friv/birthdays.cgi?month=12&amp;day=7">December 7, 1925</a></td><td class="left " data-stat="colleges"><a href="/friv/colleges.fcgi?college=stjohns">St. John's University</a></td></tr>
<tr><th class="left " data-append-csv="zawolze01" data-stat="player" scope="row"><a href="/players/z/zawolze01.html">Zeke Zawoluk</a></th><td class="right " data-stat="year_min">1953</td><td class="right " data-stat="year_max">1955</td><td class="center " data-stat="pos">F-C</td><td class="right " csk="79.0" data-stat="height">6-7</td><td class="right " data-stat